In [48]:
import numpy as np
import pandas as pd

In [49]:
def print_grid(variables):
    """
    Affiche la grille actuelle de manière lisible.

    Args:
    - variables: La grille (liste 2D) à afficher.
    """
    for row in variables:
        print(" ".join(row))
    print("\n" + "="*20 + "\n")


# CSP Class with degree heuristics

In [ ]:
class CSP2:
    def __init__(self, variables, input, domains, constraints_local, constraints_global,iterations=0):
        """Construct a CSP problem."""
        self.variables = variables  # Contient la matrice de la partie (donc matrice de variable)
        self.input = input
        self.domains = domains
        self.constraints_local = constraints_local  # Liste de toutes les contraintes locales (fonctions)
        self.constraints_global = constraints_global

        self.iterations = iterations

    # Initialiser les variables
    def initialise_variable(self, input):
        self.variables = input[3:]

    def initialise_input(self, input):
        self.input = input[:3]

    # Préserver les valeurs indices
    def initialise_assignment(self, assignment):
        for i in range(len(self.variables)):
            for j in range(len(self.variables[0])):
                if self.variables[i][j] != "0":
                    assignment[(i, j)] = self.variables[i][j]
        return assignment

    # Initialiser les domaines pour chaque variable
    def initialise_domains(self, domain):
        self.domains = {}
        for i in range(len(self.variables)):
            for j in range(len(self.variables[0])):
                self.domains[(i, j)] = domain
    
    def heuristic_MRV(self):
        """
        Minimum Remaining Values (MRV) heuristic.
        Sorts the domains of variables by the number of consistent values (smallest domain first).
        """
        new_domain = {}

        for i in range(len(self.variables)):
            for j in range(len(self.variables[0])):
                new_list = []  # Reset the list for each variable (i, j)
                for value in self.domains[(i, j)]:
                    # Add values that satisfy local constraints
                    if self.Satisfy_constraints_local(value, (i, j)):
                        new_list.append(value)
                # Update the new domain with the filtered values
                new_domain[(i, j)] = new_list

        # Sort the domains by the number of remaining values (smallest first)
        self.domains = dict(sorted(new_domain.items(), key=lambda item: len(item[1])))

        return self.domains

    # Tri un domaine selon le degrée de chaque variable
    def heuristic_degree(self,var):
        domaine_compteur=[]
        for variable in self.domains[var]:
            variable = None
            rows = len(self.variables)
            cols = len(self.variables[0])
            row=var[0]
            col=var[1]
            compteur=0
            ligne=False
            colonne=False
            #diags

            directions =        [(-1, -1), (-1, 0), (-1, 1),
                                (0, -1),         (0, 1),
                                (1, -1), (1, 0), (1, 1)]       
            
            for dr, dc in directions:
                nr, nc = row + dr, col + dc
                if 0 <= nr < rows and 0 <= nc < cols and self.variables[nr][nc]=="0":
                    if(dr==1 and dc==0 and variable=="^"):
                        compteur+=3
                        colonne=True
                    elif(dr==-1 and dc==0 and variable=="v"):
                        compteur+=3
                        colonne=True
                    elif(dr==0 and dc==1 and variable=="<"):
                        compteur+=3
                        ligne=True
                    elif(dr==0 and dc==-1 and variable==">"):
                        compteur+=3
                        ligne=True
                    if(variable=="M"):
                        if(ligne or colonne):
                            compteur+=2
                        else:
                            compteur+=4

            if(variable=="."):
                compteur+=5 
            # Toujours le minimum
            if(compteur=="S"):
                compteur=-1   

            domaine_compteur.append(compteur)  

        associe = zip(domaine_compteur,self.domains[var])

        # Étape 2 : Trier selon liste2
        associe_trie = sorted(associe)  # Trie par défaut selon les premiers éléments (liste2)

        # Étape 3 : Séparer les listes triées
        liste2_trie, liste1_trie = zip(*associe_trie)  

        self.domains[var] = list(liste1_trie)     
                
            

    
    
    def select_unnasigned_variable(self, assignment):
        """Sélectionne une variable non assignée."""
        for cle in self.domains:
            if cle not in assignment:
                return cle
        return None  # Toutes les variables ont été assignées

    # Backtracking récursif
    def recursive_backtracking(self, assignment=None):
        """
        Résout le CSP en utilisant le backtracking récursif.
        """
        self.iterations+=1
        if assignment is None:
            assignment = {}  # Initialisation de l'assignation

        # Vérifie si toutes les variables ont été assignées
        if len(assignment) == len(self.variables) * len(self.variables[0]):
            if self.Satisfy_constrains_global():  # Vérifie les contraintes globales
                return assignment  # Solution trouvée
            else:
                return None  # Solution invalide

        # Sélectionne une variable non assignée
        var = self.select_unnasigned_variable(assignment)
        self.heuristic_degree(var)
       
        # Parcourt les valeurs du domaine de la variable
        for value in self.domains[var]:
            # Vérifie si la valeur est cohérente avec les contraintes locales
            if self.Satisfy_constraints_local(value, var):

                # Assigne la valeur à la variable
                assignment[var] = value
                row, col = var
                self.variables[row][col] = value

                # Affiche la grille mise à jour
                #print_grid(self.variables)
                

                # Appel récursif
                result = self.recursive_backtracking(assignment)

                # Si une solution est trouvée
                if result is not None:
                    return result

                # Backtrack : désassignation
                del assignment[var]
                self.variables[row][col] = "0"

        # Si aucune solution n'est trouvée
        return None

    # Vérifie les contraintes locales
    def Satisfy_constraints_local(self, value, variable):
        for i in range(len(self.constraints_local)):
            if not self.constraints_local[i](self.input, self.variables, variable, value):
                return False
        return True

    # Vérifie les contraintes globales
    def Satisfy_constrains_global(self):
        for i in range(len(self.constraints_global)):
            if not self.constraints_global[i](self.input, self.variables):
                return False
        return True


# CSP Class without degree heuristics

In [51]:
class CSP:
    def __init__(self, variables, input, domains, constraints_local, constraints_global,iterations=0):
        """Construct a CSP problem."""
        self.variables = variables  # Contient la matrice de la partie (donc matrice de variable)
        self.input = input
        self.domains = domains
        self.constraints_local = constraints_local  # Liste de toutes les contraintes locales (fonctions)
        self.constraints_global = constraints_global

        self.iterations = iterations

    # Initialiser les variables
    def initialise_variable(self, input):
        self.variables = input[3:]

    def initialise_input(self, input):
        self.input = input[:3]

    # Préserver les valeurs indices
    def initialise_assignment(self, assignment):
        for i in range(len(self.variables)):
            for j in range(len(self.variables[0])):
                if self.variables[i][j] != "0":
                    assignment[(i, j)] = self.variables[i][j]
        return assignment

    # Initialiser les domaines pour chaque variable
    def initialise_domains(self, domain):
        self.domains = {}
        for i in range(len(self.variables)):
            for j in range(len(self.variables[0])):
                self.domains[(i, j)] = domain
    
    def heuristic_MRV(self):
        """
        Minimum Remaining Values (MRV) heuristic.
        Sorts the domains of variables by the number of consistent values (smallest domain first).
        """
        new_domain = {}

        for i in range(len(self.variables)):
            for j in range(len(self.variables[0])):
                new_list = []  # Reset the list for each variable (i, j)
                for value in self.domains[(i, j)]:
                    # Add values that satisfy local constraints
                    if self.Satisfy_constraints_local(value, (i, j)):
                        new_list.append(value)
                # Update the new domain with the filtered values
                new_domain[(i, j)] = new_list

        # Sort the domains by the number of remaining values (smallest first)
        self.domains = dict(sorted(new_domain.items(), key=lambda item: len(item[1])))

        return self.domains               
            

    
    
    def select_unnasigned_variable(self, assignment):
        """Sélectionne une variable non assignée."""
        for cle in self.domains:
            if cle not in assignment:
                return cle
        return None  # Toutes les variables ont été assignées

    # Backtracking récursif
    def recursive_backtracking(self, assignment=None):
        """
        Résout le CSP en utilisant le backtracking récursif.
        """
        self.iterations+=1
        if assignment is None:
            assignment = {}  # Initialisation de l'assignation

        # Vérifie si toutes les variables ont été assignées
        if len(assignment) == len(self.variables) * len(self.variables[0]):
            if self.Satisfy_constrains_global():  # Vérifie les contraintes globales
                return assignment  # Solution trouvée
            else:
                return None  # Solution invalide

        # Sélectionne une variable non assignée
        var = self.select_unnasigned_variable(assignment)
               
        # Parcourt les valeurs du domaine de la variable
        for value in self.domains[var]:
            # Vérifie si la valeur est cohérente avec les contraintes locales
            if self.Satisfy_constraints_local(value, var):

                # Assigne la valeur à la variable
                assignment[var] = value
                row, col = var
                self.variables[row][col] = value

                # Affiche la grille mise à jour
                #print_grid(self.variables)
                

                # Appel récursif
                result = self.recursive_backtracking(assignment)

                # Si une solution est trouvée
                if result is not None:
                    return result

                # Backtrack : désassignation
                del assignment[var]
                self.variables[row][col] = "0"

        # Si aucune solution n'est trouvée
        return None

    # Vérifie les contraintes locales
    def Satisfy_constraints_local(self, value, variable):
        for i in range(len(self.constraints_local)):
            if not self.constraints_local[i](self.input, self.variables, variable, value):
                return False
        return True

    # Vérifie les contraintes globales
    def Satisfy_constrains_global(self):
        for i in range(len(self.constraints_global)):
            if not self.constraints_global[i](self.input, self.variables):
                return False
        return True


# Local Constraints

Create local constraints that we will look only when we are trying to find the domains for each variables

In [52]:
def constraint_Row_Columns(input,variables,variable,curent_variable):
    """
        variable est un tuple (row,column)
    """
    curent_variable_row=variable[0]
    curent_variable_column = variable[1]

    count_rows_zeros=0
    countrows = 0
    for row in range(len(variables)):
        if(variables[row][curent_variable_column] not in [".","0"] and row!=curent_variable_row):
            countrows+=1
        if(variables[row][curent_variable_column] =="0"and row!=curent_variable_row):
            count_rows_zeros+=1
    
    count_columns_zeros=0
    countcolumns = 0
    for column in range(len(variables[0])):
        if(variables[curent_variable_row][column] not in [".","0"] and column!=curent_variable_column):
            countcolumns+=1
        if(variables[curent_variable_row][column] =="0" and column!=curent_variable_column):
            count_columns_zeros+=1
    
    # Ne pas dépasser le nb de bateau max
    if(curent_variable!="."):
        if(countrows+1>input[0][curent_variable_column]):
                return False
        if(countcolumns+1>input[1][curent_variable_row]):
                return False
    # Ne pas être en dessous du nb de bateau
    else : 
        if(count_rows_zeros+countrows<input[0][curent_variable_column]):
            return False
        if(count_columns_zeros+countcolumns<input[1][curent_variable_row]):
            return False
    return True
        


# Only for the submarines because it could put a lot of them in the grid

def fleet_constraint(input,variables,variable,curent_variable):
    """
        variable est un tuple (row,column)
    """
    curent_variable_row=variable[0]
    curent_variable_column = variable[1]
    if(curent_variable=="S"):
        countS=0
        for i in range(len(variables)):
            for j in range(len(variables[0])):
                if(curent_variable_row!=i or curent_variable_column!=j):
                    if(variables[i][j]=='S'):
                        countS+=1
        return countS<=input[2][0]    

    return True

def local_position_constraint(input, variables, variable, curent_variable):
    rows = len(variables)
    cols = len(variables[0])

    curent_variable_row=variable[0]
    curent_variable_column = variable[1]
     

    if(curent_variable=="<"):
        if(curent_variable_column==cols-1):return False
        # On gère la variable à droite initiale
        if(variables[curent_variable_row][curent_variable_column+1] not in ["M",">","0"]):
            return False
        
        
        # Pour le < on a donc enlever la direction de la droite
        directions = [(-1, -1), (-1, 0), (-1, 1),
                        (0, -1),           
                        (1, -1), (1, 0), (1, 1)]
        for dr, dc in directions:
            nr, nc = curent_variable_row + dr, curent_variable_column + dc
            if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] not in ['.','0']:
                return False
            
        curent_variable_column=curent_variable_column+1    
        
        # On c'est que c'est peut être la fin du bateau ou encore une partie ou rien 
        if(variables[curent_variable_row][curent_variable_column]=="0"):    
            # Pour le > on a donc enlever la direction de la gauche
            directions = [        (-1, 0), (-1, 1),
                                            (0, 1),
                                   (1, 0), (1, 1)]
            for dr, dc in directions:
                nr, nc = curent_variable_row + dr, curent_variable_column + dc
                if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] not in ['.','0']:
                    if(dr==0 and dc==1 and variables[nr][nc] in ["M",">"]):
                        pass
                    else:
                        return False
    
    elif(curent_variable==">"):
        
        if(curent_variable_column==0):return False  
        # On gère la variable à gauche initiale
        if(variables[curent_variable_row][curent_variable_column-1]not in ["M","<","0"]):
            return False      
        
        
        # Pour le < on a donc enlever la direction de la droite
        directions = [(-1, -1), (-1, 0), (-1, 1),
                                         (0, 1),           
                        (1, -1), (1, 0), (1, 1)]
        for dr, dc in directions:
            nr, nc = curent_variable_row + dr, curent_variable_column + dc
            if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] not in ['.','0']:
                return False
            
        curent_variable_column=curent_variable_column-1  
           
            #Pour le < on a donc enlever la direction de la gauche
        if(variables[curent_variable_row][curent_variable_column]=="0"):
            directions = [(-1, -1), (-1, 0),
                      (0, -1),
                      (1, -1), (1, 0)]
             
            for dr, dc in directions:
                nr, nc = curent_variable_row + dr, curent_variable_column + dc
                if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] not in ['.','0']:
                    if(dr==0 and dc==-1 and variables[nr][nc] in ["<","M"]):
                        pass
                    else :
                        return False
                          
    elif(curent_variable=="^"):
        if(curent_variable_row==rows-1):
            return False
        if(variables[curent_variable_row+1][curent_variable_column]not in ["M","v","0"]):
            return False
        # Pour le ^ on a donc enlever la direction du bas
        directions = [(-1, -1), (-1, 0), (-1, 1),
                        (0, -1),          (0, 1),
                        (1, -1),          (1, 1)]
        for dr, dc in directions:
            nr, nc = curent_variable_row + dr, curent_variable_column + dc
            if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] not in ['.','0']:
                return False
            
        curent_variable_row=curent_variable_row+1  
       
        if(variables[curent_variable_row][curent_variable_column]=="0"):
            directions = [        
                        (0, -1),           (0, 1),
                        (1, -1),  (1, 0), (1, 1)]
             
            for dr, dc in directions:
                nr, nc = curent_variable_row + dr, curent_variable_column + dc
                if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] not in ['.','0']:
                    if(dr==1 and dc==0 and variables[nr][nc] in ["v","M"]):
                        pass
                    else :
                        return False

    elif(curent_variable=="v"):
        if(curent_variable_row==0):
            return False
        if(variables[curent_variable_row-1][curent_variable_column]not in ["M","^","0"]):
            return False
        # Pour le v on a donc enlever la direction du bas
        directions = [  (-1, -1),         (-1, 1),
                        (0, -1),          (0, 1),
                        (1, -1), (1, 0),  (1, 1)]
        for dr, dc in directions:
            nr, nc = curent_variable_row + dr, curent_variable_column + dc
            if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] not in ['.','0']:
                return False
            
        curent_variable_row=curent_variable_row-1
       
        if(variables[curent_variable_row][curent_variable_column]=="0"):    
            
            directions = [(-1, -1),  (-1, 0), (-1, 1),
                        (0, -1),              (0, 1)
                                                    ]
             
            for dr, dc in directions:
                nr, nc = curent_variable_row + dr, curent_variable_column + dc
                if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] not in ['.','0']:
                    if(dr==-1 and dc==0 and variables[nr][nc] in ["^","M"]):
                        pass
                    else :
                        return False

    elif(curent_variable=="S"):
        directions = [(-1, -1), (-1, 0), (-1, 1),
                      (0, -1),         (0, 1),
                      (1, -1), (1, 0), (1, 1)]
        for dr, dc in directions:
            nr, nc = curent_variable_row + dr, curent_variable_column + dc
            if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] not in ['.','0']:
                return False
            
    elif(curent_variable=="M"):
        directions = [(-1, -1),        (-1, 1),
        
                      (1, -1),          (1, 1)]
        # Diagonales
        for dr, dc in directions:
            nr, nc = curent_variable_row + dr, curent_variable_column + dc
            if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] not in ["0","."]:
                return False

        directions = [         (-1,0),
                       (0,-1),        (0,1),
                                (1,0)          ]
        Nord=False
        Nord_point=False
        Sud=False
        Sud_point=False
        Est=False
        Est_point=False
        Ouest=False
        Ouest_point=False

        
        for dr, dc in directions:
            nr, nc = curent_variable_row + dr, curent_variable_column + dc
            if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc]!="0":
                # Haut
                if(dr==-1 and dc==0):
                    if(variables[nr][nc] in ["^","M"]):
                        Nord=True
                        if(nr+2==rows):return False
                    elif(variables[nr][nc]=="."):
                        Nord_point=True
                    else:
                        return False
                    
                #Bas
                if(dr==1 and dc==0):
                    if(variables[nr][nc] in ["v","M"]):
                        Sud=True
                        if(nr-2==-1):return False
                    elif(variables[nr][nc]=="."):
                        Sud_point_point=True
                    else:
                        return False
                    
                # Droite
                elif(dr==0 and dc==1):
                    if(variables[nr][nc] in [">","M"]):
                        Est=True
                        if(nc-2==-1):return False
                    elif(variables[nr][nc]=="."):
                        Est_point=True
                    else:
                        return False
                    
                # Gauche   
                elif(dr==0 and dc==-1):
                    if(variables[nr][nc] in ["<","M"]):
                        Ouest=True
                        if(nc+2==cols):return False
                    elif(variables[nr][nc]=="."):
                        Ouest_point=True
                    else:
                        return False

                if((Nord_point or Sud_point) and (Ouest_point or Est_point)):
                    return False
                
                if((Nord or Sud) and (Est or Ouest)):
                    return False
                
                if((Nord_point and Sud) or (Sud_point and Nord) or (Est_point and Ouest) or (Ouest_point and Est)):
                    return False
                
                
    
    elif(curent_variable=="."):
        if(0<=curent_variable_row-1 and variables[curent_variable_row-1][curent_variable_column]=="^"):
            return False
        if(curent_variable_row+1<rows and variables[curent_variable_row+1][curent_variable_column]=="v"):
            return False
        if(0<=curent_variable_column-1 and variables[curent_variable_row][curent_variable_column-1]=="<"):
            return False
        if(curent_variable_column+1<cols and variables[curent_variable_row][curent_variable_column+1]==">"):
            return False

    return True



# Global Constraints

Creation of the global constraints

In [53]:
def Global_constraint_Row_Columns(input, variables):
    # Comptage par ligne : compter tout sauf '.'
    non_dots_per_row = []
    for row in variables:
        count = 0
        for char in row:
            if char != '.':
                count += 1
        non_dots_per_row.append(count)

    # Comptage par colonne : compter tout sauf '.'
    non_dots_per_col = []
    for col in range(len(variables[0])):
        count = 0
        for row in variables:
            if row[col] != '.':
                count += 1
        non_dots_per_col.append(count)

    # Vérifier que les contraintes sont respectées
    for a, b in zip(input[0], non_dots_per_col):
        if a < b:
            return False

    for a, b in zip(input[1], non_dots_per_row):
        if a < b:
            return False

    return True


# Only for the submarines because it could put a lot of them in the grid
def Global_fleet_constraint(input, variables):
    fleet_constraint_variable = input[2]

    nb_submarines = sum(row.count('S') for row in variables)
    nb_destroyers = 0
    nb_carriers = 0
    nb_battleships = 0

    for row in range(len(variables)):
        for col in range(len(variables[0])):
            compteur = 0  # Reset the counter for each ship found

            # Horizontal ship check (starting with '<')
            if variables[row][col] == "<":
                current_row = row
                current_col = col + 1
                # Check if the ship is properly filled with "M" and ends with ">"
                while current_col < len(variables[0]) and variables[current_row][current_col] == "M":
                    compteur += 1
                    current_col += 1

                # Ensure the ship ends with ">"
                if current_col == len(variables[0]) or variables[current_row][current_col] != ">":
                    return False  # Ship is not properly closed
                else:
                    if compteur == 0: 
                        nb_destroyers += 1
                    elif compteur == 1: 
                        nb_carriers += 1
                    elif compteur == 2: 
                        nb_battleships += 1

            # Vertical ship check (starting with '^')
            elif variables[row][col] == "^":
                current_row = row + 1
                current_col = col
                # Check if the ship is properly filled with "M" and ends with "v"
                while current_row < len(variables) and variables[current_row][current_col] == "M":
                    compteur += 1
                    current_row += 1

                # Ensure the ship ends with "v"
                if current_row == len(variables) or variables[current_row][current_col] != "v":
                    return False  # Ship is not properly closed
                else:
                    if compteur == 0:
                        nb_destroyers += 1
                    elif compteur == 1:
                        nb_carriers += 1
                    elif compteur == 2:
                        nb_battleships += 1

    # Check if the number of ships matches the fleet constraints
    if nb_submarines != fleet_constraint_variable[0]: 
        return False
    if nb_destroyers != fleet_constraint_variable[1]: 
        return False
    if nb_carriers != fleet_constraint_variable[2]: 
        return False
    if nb_battleships != fleet_constraint_variable[3]: 
        return False

    return True


# J'ai mis input pour que cela fonctionne lors de l'appel de la fonction
def Global_constraint_Touch_Other_ships(input,variables):
    rows, cols = len(variables), len(variables[0])
    for row in range(rows):
        for col in range(cols):
            if(variables[row][col]=="<"):
                # Pour le < on a donc enlever la direction de la droite
                directions = [(-1, -1), (-1, 0), (-1, 1),
                              (0, -1),         
                              (1, -1), (1, 0), (1, 1)]
                for dr, dc in directions:
                    nr, nc = row + dr, col + dc
                    if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] != '.':
                        return False
                
                curent_col=col+1
                directions = [(-1, -1), (-1, 0), (-1, 1),
                                 
                              (1, -1), (1, 0), (1, 1)]
                if(curent_col==cols):return False

                # On vérifie pour tout les M du bateau (on a donc enlevé les directions vers la gauche et la droite)
                while(variables[row][curent_col]!=">"):                                                           
                    for dr, dc in directions:
                        nr, nc = row + dr, curent_col + dc
                        if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] != '.':
                            return False
                    curent_col+=1
                    if(curent_col==cols):return False
                
                # Pour le > on a donc enlever la direction de la gauche
                directions = [(-1, -1), (-1, 0), (-1, 1),
                                                 (0, 1),
                               (1, -1), (1, 0), (1, 1)]
                for dr, dc in directions:
                    nr, nc = row + dr, curent_col + dc
                    if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] != '.':
                        return False

                
            elif(variables[row][col]=="^"):
                # Pour le ^ on a donc enlever la direction du bas
                directions = [(-1, -1), (-1, 0), (-1, 1),
                              (0, -1),          (0, 1),
                              (1, -1),          (1, 1)]
                for dr, dc in directions:
                    nr, nc = row + dr, col + dc
                    if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] != '.':
                        return False


                # On vérifie pour tout les M du bateau (on a donc enlevé les directions vers le haut et le bas)
                directions = [(-1, -1),        (-1, 1),
                              (0, -1),         (0, 1),
                              (1, -1),         (1, 1)]
                
                curent_row=row+1 # Si on est déja en dehors da la matrice c'est que c'est faux
                if(curent_row==rows):return False
                while(variables[curent_row][col]!="v"):                                                           
                    for dr, dc in directions:
                        nr, nc = curent_row + dr, col + dc
                        if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] != '.':
                            return False
                    curent_row+=1
                    if(curent_row==rows):return False
                    


                # Pour le v on a donc enlever la direction du haut
                directions = [(-1, -1),        (-1, 1),
                              (0, -1),         (0, 1),
                              (1, -1), (1, 0), (1, 1)]
                for dr, dc in directions:
                    nr, nc = curent_row + dr, col + dc
                    if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] != '.':
                        return False
                        

            elif(variables[row][col]=="S"):
                directions = [(-1, -1), (-1, 0), (-1, 1),
                              (0, -1),         (0, 1),
                              (1, -1), (1, 0), (1, 1)]
                for dr, dc in directions:
                    nr, nc = row + dr, col + dc
                    if 0 <= nr < rows and 0 <= nc < cols and variables[nr][nc] != '.':
                        return False
    return True



# Result Without Heuristic

In [54]:
def main():
    matrix = [
        [2, 2, 1, 2, 1, 2],
        [4, 0, 1, 3, 1, 1],
        [3, 2, 1, 0],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", 'M', "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
    ]


    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"] 
    assignment = {}

    # Initialize CSP object
    csp = CSP(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f"nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
< > . < > .
. . . . . .
. . . . . S
. < M > . .
. . . . . S
S . . . . .


nb iterations 4500 


In [55]:
def main():
    
    matrix = [
    [3, 1, 2, 2, 0, 2],
    [0, 4, 1, 3, 1, 1],
    [3, 2, 1, 0],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["v", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"]
    ]

    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)

    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f" nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
. . . . . .
^ . < > . S
M . . . . .
v . < > . .
. . . . . S
. S . . . .


 nb iterations 169 


In [56]:
def main():
    
    matrix = [
    [3, 1, 1, 2, 1, 2 ],
    [2, 1, 3, 0, 4, 0 ],
    [3, 2, 1, 0 ],
    ["0", "0", "0", "<", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"]
]


    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    print(f"nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
. . . < > .
^ . . . . .
v . . S . S
. . . . . .
< M > . . S
. . . . . .


nb iterations 346 


## Score without Heuristics
4500

169

346

# Result With MRV Heuristics

In [57]:
def main():
    matrix = [
        [2, 2, 1, 2, 1, 2],
        [4, 0, 1, 3, 1, 1],
        [3, 2, 1, 0],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", 'M', "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
    ]


    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    csp.heuristic_MRV()
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f"nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
< > . < > .
. . . . . .
. . . . . S
. < M > . .
. . . . . S
S . . . . .


nb iterations 3490 


In [58]:
def main():
    
    matrix = [
    [3, 1, 2, 2, 0, 2],
    [0, 4, 1, 3, 1, 1],
    [3, 2, 1, 0],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["v", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"]
    ]

    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    csp.heuristic_MRV()
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f" nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
. . . . . .
^ . < > . S
M . . . . .
v . < > . .
. . . . . S
. S . . . .


 nb iterations 96 


In [59]:
def main():
    
    matrix = [
    [3, 1, 1, 2, 1, 2 ],
    [2, 1, 3, 0, 4, 0 ],
    [3, 2, 1, 0 ],
    ["0", "0", "0", "<", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"]
]


    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    csp.heuristic_MRV()
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    print(f"nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
. . . < > .
^ . . . . .
v . . S . S
. . . . . .
< M > . . S
. . . . . .


nb iterations 124 


## Score With MRV Heuristics

3490

96

124

# Results with Degree Heuristics

In [60]:
def main():
    matrix = [
        [2, 2, 1, 2, 1, 2],
        [4, 0, 1, 3, 1, 1],
        [3, 2, 1, 0],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", 'M', "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
    ]


    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP2(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f"nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
< > . < > .
. . . . . .
. . . . . S
. < M > . .
. . . . . S
S . . . . .


nb iterations 3225 


In [61]:
def main():
    
    matrix = [
    [3, 1, 2, 2, 0, 2],
    [0, 4, 1, 3, 1, 1],
    [3, 2, 1, 0],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["v", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"]
    ]

    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP2(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f" nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
. . . . . .
^ . < > . S
M . . . . .
v . < > . .
. . . . . S
. S . . . .


 nb iterations 432 


In [62]:
def main():
    
    matrix = [
    [3, 1, 1, 2, 1, 2 ],
    [2, 1, 3, 0, 4, 0 ],
    [3, 2, 1, 0 ],
    ["0", "0", "0", "<", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"]
]


    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    print(f"nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
. . . < > .
^ . . . . .
v . . S . S
. . . . . .
< M > . . S
. . . . . .


nb iterations 346 


## Score With degree Heuristics

3225

432

346

# Results with MRV Heuristics and domain modification

In [63]:
def main():
    matrix = [
        [2, 2, 1, 2, 1, 2],
        [4, 0, 1, 3, 1, 1],
        [3, 2, 1, 0],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", 'M', "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
    ]


    domain = ["<", ">", "^", "v", ".","S","M"]
    #domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    csp.heuristic_MRV()
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f"nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
< > . < > .
. . . . . .
. . . . . S
. < M > . .
. . . . . S
S . . . . .


nb iterations 183 


In [64]:
def main():
    
    matrix = [
    [3, 1, 2, 2, 0, 2],
    [0, 4, 1, 3, 1, 1],
    [3, 2, 1, 0],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["v", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"]
    ]

    domain = ["<", ">", "^", "v", ".","S","M"]
    #domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    csp.heuristic_MRV()
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f" nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
. . . . . .
^ . < > . S
M . . . . .
v . < > . .
. . . . . S
. S . . . .


 nb iterations 103 


In [65]:
def main():
    
    matrix = [
    [3, 1, 1, 2, 1, 2 ],
    [2, 1, 3, 0, 4, 0 ],
    [3, 2, 1, 0 ],
    ["0", "0", "0", "<", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"]
]


    domain = ["<", ">", "^", "v", ".","S","M"]
    #domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    csp.heuristic_MRV()
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    print(f"nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
. . . < > .
^ . . . . .
v . . S . S
. . . . . .
< M > . . S
. . . . . .


nb iterations 67 


## Score with MRV heuristics and domain modification

183

103

67

# Results with MRV heuristics and Degree heuristics

In [66]:
def main():
    matrix = [
        [2, 2, 1, 2, 1, 2],
        [4, 0, 1, 3, 1, 1],
        [3, 2, 1, 0],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", 'M', "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
        ["0", "0", "0", "0", "0", "0"],
    ]


    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP2(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    csp.heuristic_MRV()
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f"nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
< > . < > .
. . . . . .
. . . . . S
. < M > . .
. . . . . S
S . . . . .


nb iterations 3353 


In [67]:
def main():
    
    matrix = [
    [3, 1, 2, 2, 0, 2],
    [0, 4, 1, 3, 1, 1],
    [3, 2, 1, 0],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["v", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"]
    ]

    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP2(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    csp.heuristic_MRV()
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f" nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
. . . . . .
^ . < > . S
M . . . . .
v . < > . .
. . . . . S
. S . . . .


 nb iterations 84 


In [68]:
def main():
    
    matrix = [
    [3, 1, 1, 2, 1, 2 ],
    [2, 1, 3, 0, 4, 0 ],
    [3, 2, 1, 0 ],
    ["0", "0", "0", "<", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"],
    ["0", "0", "0", "0", "0", "0"]
]


    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP2(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    csp.heuristic_MRV()
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    print(f"nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
. . . < > .
^ . . . . .
v . . S . S
. . . . . .
< M > . . S
. . . . . .


nb iterations 823 


## Score with MRV heuristics and Degree heuristics

3353

84

823

It's useless to try Degree heuristics with domain modification because the degree heuristics will change the domain all the time. 

After doing all these tests it looks like degree heuristics is not really helping our case.


However let's compare degree + MRV and domain modification + MRV with a bigger matrix

# Results for a 8*8 matrix with MRV Heuristics and domain modification

In [72]:
def main():
    
    matrix = [
    [2,2,5,0,5,0,1,4],
    [4,2,4,1,2,1,2,3],
    [3,3,2,1],
    ["0", "0", "0", "0", "0", "0","0", "0"],
    ["0", "0", "0", "0", "0", "0","0", "0"],
    ["0", "0", "0", "0", "0", "0","0", "0"],
    ["0", "0", "0", "0", "0", "0","0", "0"],
    ["0", "0", "0", "0", "v", "0","0", "0"],
    ["0", "0", "0", "0", "0", "0","0", "0"],
    ["0", "0", "0", "0", "0", "0","<", "0"],   
    [".", "0", "0", "0", "0", "0","0", "0"],
    
    ]

    domain = ["<", ">", "^", "v", ".","S","M"]
    #domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    csp.heuristic_MRV()
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f" nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
< M > . . . . ^
. . . . ^ . . M
S . S . M . . v
. . . . M . . .
. . ^ . v . . .
. . v . . . . .
. . . . . . < >
. < > . S . . .


 nb iterations 314641 


# Results for a 8*8 matrix with MRV heuristics and Degree heuristics

In [70]:
def main():
    
    matrix = [
    [2,2,5,0,5,0,1,4],
    [4,2,4,1,2,1,2,3],
    [3,3,2,1],
    ["0", "0", "0", "0", "0", "0","0", "0"],
    ["0", "0", "0", "0", "0", "0","0", "0"],
    ["0", "0", "0", "0", "0", "0","0", "0"],
    ["0", "0", "0", "0", "0", "0","0", "0"],
    ["0", "0", "0", "0", "v", "0","0", "0"],
    ["0", "0", "0", "0", "0", "0","0", "0"],
    ["0", "0", "0", "0", "0", "0","<", "0"],   
    [".", "0", "0", "0", "0", "0","0", "0"],
    
    ]

    #domain = ["<", ">", "^", "v", ".","S","M"]
    domain = ["S", ".", "<", ">", "^","v","M"]
    assignment = {}

    # Initialize CSP object
    csp = CSP2(None, None,None, [constraint_Row_Columns,fleet_constraint,local_position_constraint], [Global_constraint_Row_Columns, Global_fleet_constraint,Global_constraint_Touch_Other_ships])
    csp.initialise_variable(matrix)
    csp.initialise_input(matrix)
    csp.initialise_domains(domain)
    csp.heuristic_MRV()
    assignment=csp.initialise_assignment(assignment)

    # Solve the CSP
    solution = csp.recursive_backtracking(assignment)

    # Print the solution (if found)
    if solution:
        print("Solution found")
        print_grid(csp.variables)
    else:
        print("No solution found.")
    
    print(f" nb iterations {csp.iterations} ")


if __name__ == "__main__":
    main()

Solution found
< M > . . . . ^
. . . . ^ . . M
S . S . M . . v
. . . . M . . .
. . ^ . v . . .
. . v . . . . .
. . . . . . < >
. < > . S . . .


 nb iterations 78310 


## Score for 8*8 : 

Degree : 78310

Domain modification : 314641

As we can see, if we take a bigger matrix, degree heuristics is a good heuristics, better than domain modification